In [1]:
import praw
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
from collections import Counter
from wordcloud import WordCloud
import glob, nltk
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import datetime as dt
import nltk
nltk.download('wordnet')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
import string
nltk.download('stopwords')
import numpy as np
from textblob import TextBlob
from nltk.tokenize import word_tokenize, RegexpTokenizer
import functools
from string import digits
import yfinance as yf
import requests
import bs4 as bs

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/matteoferrazzi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/matteoferrazzi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteoferrazzi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
STOPWORDS = set(stopwords.words('english'))

In [ ]:
#Definition of the functions we will use to clean the text. We used the ones implemented in exercise sessions
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_digits_lengthword(text):
    return " ".join([word for word in str(text).split() if len(word)<20 and len(word)>2 and not word.isdigit()])
                     
def lemmatize(text):
    le = WordNetLemmatizer()
    return " ".join([le.lemmatize(word) for word in str(text).split()])

In [ ]:
#Create a table with some features of the s&p500 such as symbol, sector and name of each stock
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]

In [ ]:
#Definition of a function to download submissions and the selftext from reddit searching for a query 
#with and to clean them using the functions implemented above. If the symbol of the stock is something 
#that can be found in reddit but not referring at that stock
def searchsymbol(df,subreddit,type):
    clean_comments1 = []
    date = []
    sector = []
    name = []
    for i in range(len(df['Symbol'])):
        print(df['Symbol'][i])
        search_query = df['Symbol'][i]
        if df['Symbol'][i] == 'MMM':
            search_query = '3M'
        if df['Symbol'][i] == 'T':
            search_query = 'AT&T'
        if df['Symbol'][i] == 'A':
            search_query = 'Agilent'
        if df['Symbol'][i] == 'C':
            search_query = 'Citi'
        if df['Symbol'][i] == 'D':
            search_query = 'Dominion Energy'
        if df['Symbol'][i] == 'F':
            search_query = 'Ford'
        if df['Symbol'][i] == 'J':
            search_query = 'Jacobs'
        if df['Symbol'][i] == 'K':
            search_query = 'Kellogg\'s'
        if df['Symbol'][i] == 'L':
            search_query = 'Loews'
        if df['Symbol'][i] == 'V':
            search_query = 'Visa'
        if df['Symbol'][i] == 'O':
            search_query = 'Realty Income'
        if df['Symbol'][i] == 'IT':
            search_query = 'Gartner'
        if df['Symbol'][i] == 'ON':
            search_query = 'ON Semiconductor'
        if df['Symbol'][i] == 'TT':
            search_query = 'Trane'
        if df['Symbol'][i] == 'TECH':
            search_query = 'Bio-Techne'
        submissions = subreddit.search(search_query,sort=type,limit=None,time_filter='year')

        for submission in submissions:
            clean_text1 = submission.title+submission.selftext
            date.append(dt.datetime.utcfromtimestamp(submission.created_utc).date())
            clean_text1 = clean_text1.lower()
            clean_text1 = remove_urls(clean_text1)
            clean_text1 = remove_punctuation(clean_text1)
            clean_text1 = remove_stopwords(clean_text1)
            clean_text1 = remove_digits_lengthword(clean_text1)
            clean_text1 = lemmatize(clean_text1)
            clean_comments1.append(clean_text1)
            sector.append(df['GICS Sector'][i])
            name.append(df['Symbol'][i])
    return clean_comments1,date,sector,name

In [ ]:
#Same function as before but looking for the name of the stock a squery
def searchsecurity(df,subreddit,type):
    clean_comments1 = []
    date = []
    name = []
    sector = []

    for i in range(len(df['Security'])):
        print(df['Security'][i])
        search_query = df['Security'][i]

        submissions = subreddit.search(search_query,sort=type,limit=None,time_filter='year')

        # Loop through comments and clean the data
        for submission in submissions:
            # Clean the comment text by removing punctuation, numbers, and special characters
            clean_text1 = submission.title+submission.selftext
            date.append(dt.datetime.utcfromtimestamp(submission.created_utc).date())
            clean_text1 = clean_text1.lower()
            clean_text1 = remove_urls(clean_text1)
            clean_text1 = remove_punctuation(clean_text1)
            clean_text1 = remove_stopwords(clean_text1)
            clean_text1 = remove_digits_lengthword(clean_text1)
            clean_text1 = lemmatize(clean_text1)
                
            # Append the cleaned comment text to the list
            clean_comments1.append(clean_text1)
            sector.append(df['GICS Sector'][i])
            name.append(df['Symbol'][i])
    return clean_comments1,date,sector,name

In [ ]:
#Create a Reddit instance to download 
reddit = praw.Reddit(client_id='GpASx1ZazRoHcNZNDdEIOQ',
                     client_secret='TdxMpCPbbb3KTihf0WDXegjujgPxSg',
                     user_agent='econ/1.0 by testamatta21')

#
subreddit = reddit.subreddit('WallStreetBets')

#Search using the symbol:
#Search by relevance submissions on the subreddit
clean_commentsrel1,daterel1,namerel1,sectorrel1=searchsymbol(df,subreddit,'relevance')
#Search by hot submissions on the subreddit
clean_commentshot1,datehot1,namehot1,sectorhot1=searchsymbol(df,subreddit,'hot')
#Search by top submissions on the subreddit
clean_commentstop1,datetop1,nametop1,sectortop1=searchsymbol(df,subreddit,'top')
#Search by new submissions on the subreddit
clean_commentsnew1,datenew1,namenew1,sectornew1=searchsymbol(df,subreddit,'new')

In [ ]:
#Search using the security:
#Search by relevance submissions on the subreddit
clean_commentsrel2,daterel2,namerel2,sectorrel2=searchsecurity(df,subreddit,'relevance')
#Search by hot submissions on the subreddit
clean_commentshot2,datehot2,namehot2,sectorhot2=searchsecurity(df,subreddit,'hot')
#Search by top submissions on the subreddit
clean_commentstop2,datetop2,nametop2,sectortop2=searchsecurity(df,subreddit,'top')
#Search by new submissions on the subreddit
clean_commentsnew2,datenew2,namenew2,sectornew2=searchsecurity(df,subreddit,'new')

In [ ]:
#Add the results together
clean_comments1=clean_commentsrel1+clean_commentshot1+clean_commentstop1+clean_commentsnew1+clean_commentsrel2+clean_commentshot2+clean_commentstop2+clean_commentsnew2
sector=sectorrel1+sectorhot1+sectortop1+sectornew1+sectorrel2+sectorhot2+sectortop2+sectornew2
date=daterel1+datehot1+datetop1+datenew1+daterel2+datehot2+datetop2+datenew2
name=namerel1+namehot1+nametop1+namenew1+namerel2+namehot2+nametop2+namenew2

In [ ]:
#Create the dataframe removing duplicates and then save it as csv
df1_old = pd.DataFrame({'comments': clean_comments1,'date':date,'sector':name,'name':sector})
df1=df1_old.drop_duplicates()
df1.to_csv('data1final.csv')

In [ ]:
data1=pd.read_csv('data1final.csv')
data1["comments"] = data1['comments'].str.replace('[^\w\s]','')
data1[list([type(data1.comments[i])==float for i in range(len(data1.comments))])]
data1=data1.drop(data1[list([type(data1.comments[i])==float for i in range(len(data1.comments))])].index)